# Data Cleansing - FAO Statistics for Global Tea Production

## Data Source

The dataset is obtained from FAOSTAT which provides free access to food and agriculture data for over 245 countries and territories. It contains tea production quantity globally from 2009 and 2018.

You may download the data from this URL http://www.fao.org/faostat/en/?#data/QC

## Data Cleansing Procedure

Below are the steps taken to cleanse the data before importing to Tableau for visualisation.

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
tea_data = pd.read_csv('C:/Users/Lenovo/Downloads/FAOSTAT_data_tea.csv')
tea_data.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QC,Crops,9,Argentina,5510,Production,667,Tea,2009,2009,tonnes,71715.0,NaN,Official data
1,QC,Crops,9,Argentina,5510,Production,667,Tea,2010,2010,tonnes,92417.0,NaN,Official data
2,QC,Crops,9,Argentina,5510,Production,667,Tea,2011,2011,tonnes,92892.0,NaN,Official data
3,QC,Crops,9,Argentina,5510,Production,667,Tea,2012,2012,tonnes,82813.0,NaN,Official data
4,QC,Crops,9,Argentina,5510,Production,667,Tea,2013,2013,tonnes,80423.0,NaN,Official data


In [3]:
# Remove the aggregated value representing  China (Flag = 'A')
tea_data = tea_data[tea_data.Flag != 'A']

# Remove rows with unavailable tea production quantity (Flag = 'M')
tea_data = tea_data[tea_data.Flag != 'M']

In [4]:
# Drop redundant columns
tea_data = tea_data.drop(columns = ['Domain Code', 'Domain', 'Area Code', 'Element Code', 'Element', 'Item Code', 'Year Code', 'Flag', 'Flag Description'])

In [5]:
# Standardise the name of the countries to ensure consistency with the geolocation file
replace_values = ['Bolivia (Plurinational State of)', 
                  'China, mainland', 
                  'China, Taiwan Province of', 
                  'Democratic Republic of the Congo', 
                  'Iran (Islamic Republic of)', 
                  'Lao People\'s Democratic Republic', 
                  'Republic of Korea', 
                  'Russian Federation', 
                  'United Republic of Tanzania']
to_replace = ['Bolivia', 'China', 'Taiwan', 'Congo [DRC]', 'Iran', 'Laos', 'South Korea', 'Russia', 'Tanzania']

country_replace = dict(zip(replace_values, to_replace))
tea_data['Area'] = tea_data['Area'].replace(country_replace)

In [6]:
# Open Excel file containing geolocation data which has been downloaded from https://developers.google.com/public-data/docs/canonical/countries_csv
geolocation = pd.read_excel('C:/Users/Lenovo/Downloads/geolocation.xlsx', sheet_name = 'Sheet1')
geolocation.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [7]:
# Merge the tea data with geolocation data 
tea_data = tea_data.merge(geolocation, how = 'left', left_on = 'Area', right_on = 'name')
tea_data.head()

,Area,Item,Year,Unit,Value,country,latitude,longitude,name
0,Argentina,Tea,2009,tonnes,71715.0,AR,-38.416097,-63.616672,Argentina
1,Argentina,Tea,2010,tonnes,92417.0,AR,-38.416097,-63.616672,Argentina
2,Argentina,Tea,2011,tonnes,92892.0,AR,-38.416097,-63.616672,Argentina
3,Argentina,Tea,2012,tonnes,82813.0,AR,-38.416097,-63.616672,Argentina
4,Argentina,Tea,2013,tonnes,80423.0,AR,-38.416097,-63.616672,Argentina


In [8]:
# Drop additional columns & rename columns to more meaningful ones
tea_data = tea_data.drop(columns = 'name')
tea_data = tea_data.rename(columns = {'Area':'Country', 'country':'Country Code', 'latitude':'Latitude', 'longitude':'Longitude'})
tea_data.head()

,Country,Item,Year,Unit,Value,Country Code,Latitude,Longitude
0,Argentina,Tea,2009,tonnes,71715.0,AR,-38.416097,-63.616672
1,Argentina,Tea,2010,tonnes,92417.0,AR,-38.416097,-63.616672
2,Argentina,Tea,2011,tonnes,92892.0,AR,-38.416097,-63.616672
3,Argentina,Tea,2012,tonnes,82813.0,AR,-38.416097,-63.616672
4,Argentina,Tea,2013,tonnes,80423.0,AR,-38.416097,-63.616672


In [9]:
# The below commented code can be used to create different csv files with timestamps for testing only. 
#current_datetime = datetime.datetime.today().strftime('%d-%m-%Y %H-%M-%S')
#tea_data.to_csv(f'tea-{current_datetime}.csv', index = False)

In [10]:
# Write the final csv file containing clean data 
# Note: Before running the below code more than once, please remember to delete the existing csv file to avoid error.
tea_data.to_csv(f'tea.csv', index = False)